In [3]:
import requests
import json
import pandas as pd
import nltk
import numpy as np
import itertools
import config
from newspaper import Article
from newspaper import Config

In [4]:
user_agent = config.user_agent
config_news = Config()
config_news.browser_user_agent = config.user_agent

### Introduction
Walk through process
I found this topic to be interesting because there has been a lot of talk and media attention around the,formally known, Parental Rights in Education Bill or, colloquilly, Don't Say Gy Bill. There seems to be a lot of confusion as what this bill entails, but I am more importantly wanting to know how conversations around this bill are being made and if there's any biases in the language they're using.

In [5]:
#iterate through strings
request_strings = ['gay', "don't say gay", "parental rights"]

In [6]:
def news_query(strings_lst,url = "https://bing-news-search1.p.rapidapi.com/news/search" ):

    # url = "https://bing-news-search1.p.rapidapi.com/news/search"
    for s in strings_lst:
        querystring = {"q":s,"count":"100","textFormat":"Raw","safeSearch":"Off"}

        headers = {
            'x-bingapis-sdk': "true",
            'x-search-location': "Florida",
            'User-Agent': config.user_agent,
            'x-rapidapi-host': "bing-news-search1.p.rapidapi.com",
            'x-rapidapi-key': config.api_key
            }

        response = requests.request("GET", url, headers=headers, params=querystring)
        # return response.json()
        #print("QUERY:"+s, response.text)
        with open('data.json', 'w', encoding='utf-8') as f:
            json.dump(response.json(), f, ensure_ascii=False, indent=4)

In [113]:
def news_parse():
    json_data = news_query(request_strings)
    # for item in json_data:
    #     for data_item in item['data']:
    #         print data_item['url'], data_item['value']
    #df = pd.read_json('data.json')
    with open('data.json', encoding='utf-8') as json_data:
        data = json.load(json_data)
    df = pd.DataFrame(data['value'])
    # print(df)
    # print(df.shape)
    
    # print(df[['url']])
    # print(df[['url']].head(1))
    article_text = []
    for index, row in df[['url']].iterrows():
        # print(row['url'])
        url=row['url']
        article = Article(url)
        try:
            article.download()
            article.html
            article.parse()
            # print("/n")
            # print(index,article.text)
            article_text.append(article.text)
        except:
            article_text.append("error")
    # print(article_text)
    df['text'] = article_text
    print(df)
    return df

In [114]:
df = news_parse()

C:\Users\lukea\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


          _type                                               name  \
0   NewsArticle  Virginia father hits back at Randi Weingarten,...   
1   NewsArticle  Teacher’s union boss on parental rights bills:...   
2   NewsArticle  LA Times columnist: Parental rights in educati...   
3   NewsArticle  White House says DeSantis Disney moves and par...   
4   NewsArticle  More States Move to Affirm Parental Rights in ...   
..          ...                                                ...   
77  NewsArticle  Do Americans Support Florida’s New Parental Ri...   
78  NewsArticle  Highway to Health: Florida’s Parental Rights i...   
79  NewsArticle  Texas is trampling parents’ rights in its inve...   
80  NewsArticle  Gov. DeSantis signs Parental Rights in Educati...   
81  NewsArticle  Paxon students participate in walkout to prote...   

                                                  url  \
0   https://www.foxnews.com/media/virginia-father-...   
1   https://www.bizpacreview.com/2022/04/23/t

In [115]:
def clean_news_parse(df):
    # want to know name,text, provider,time
    df_new = df[['name','description','provider','datePublished','text']]
    df_provider = df_new[['provider']]
    provider_list = df_provider.values.tolist()
    for index, row in df_provider.iterrows():
        #print(df_new.iloc[index]['provider'])
        # print(row[0][0]['name'])
        df_new.iloc[index]['provider'] = row[0][0]['name']
    # text_length=[]
    # for index, row in df_new[['text']].iterrows():
    #     print(len(row))
    #     #text_length.append
    df_new['text'] = df_new['text'].astype("string")
    return df_new

In [116]:
clean_df = clean_news_parse(df)
clean_df

C:\Users\lukea\AppData\Local\Temp/ipykernel_12376/3467388693.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['text'] = df_new['text'].astype("string")


,name,description,provider,datePublished,text
0,"Virginia father hits back at Randi Weingarten,...","Loudoun County, Virginia parent Brandon Michon...",Fox News,2022-04-25T16:50:00.0000000Z,NEW You can now listen to Fox News articles! ...
1,Teacher’s union boss on parental rights bills:...,Teacher's union honcho Randi Weingarten went o...,BizPac Review,2022-04-23T10:25:00.0000000Z,
2,LA Times columnist: Parental rights in educati...,An L.A. Times columnist claimed that the GOP i...,Fox News,2022-04-20T15:00:00.0000000Z,NEW You can now listen to Fox News articles! ...
3,White House says DeSantis Disney moves and par...,The move by Florida Gov. Ron DeSantis and the ...,Washington Examiner on MSN.com,2022-04-21T18:29:00.0000000Z,© Provided by Washington Examiner The move by...
4,More States Move to Affirm Parental Rights in ...,States across the country are following Florid...,The Daily Signal,2022-04-19T17:29:00.0000000Z,Florida has been the center of national attent...
...,...,...,...,...,...
77,Do Americans Support Florida’s New Parental Ri...,They also try to get to the bottom of whether ...,FiveThirtyEight,2022-04-05T06:12:00.0000000Z,In this installment of the FiveThirtyEight Pol...
78,Highway to Health: Florida’s Parental Rights i...,"On March 28, Florida Governor Ron DeSantis sig...",The Daily Free Press,2022-04-01T02:56:00.0000000Z,"On March 28, Florida Governor Ron DeSantis sig..."
79,Texas is trampling parents’ rights in its inve...,"According to the order, parents who follow the...",Washington Post,2022-04-08T13:02:00.0000000Z,Placeholder while article actions load In rec...
80,Gov. DeSantis signs Parental Rights in Educati...,"JACKSONVILLE, Fla. — Gov. Ron DeSantis signed ...",Action News Jax,2022-03-28T21:26:00.0000000Z,"JACKSONVILLE, Fla. — Gov. Ron DeSantis signed ..."


In [117]:
def add_text_length(df):
    print(df.dtypes)
    text_length=[]
    for index, row in df.iterrows():
        print(type(row['text']))
        print(len(row['text']))
        text_length.append(len(row['text']))
    df['text_length'] = text_length
    return df
        #text_length.append

In [118]:
clean_df = add_text_length(clean_df)

name             object
description      object
provider         object
datePublished    object
text             string
dtype: object
<class 'str'>
3188
<class 'str'>
0
<class 'str'>
3935
<class 'str'>
2243
<class 'str'>
4876
<class 'str'>
5147
<class 'str'>
3647
<class 'str'>
5
<class 'str'>
3281
<class 'str'>
3066
<class 'str'>
9520
<class 'str'>
9413
<class 'str'>
0
<class 'str'>
4049
<class 'str'>
1448
<class 'str'>
3557
<class 'str'>
3617
<class 'str'>
0
<class 'str'>
188
<class 'str'>
2383
<class 'str'>
12427
<class 'str'>
2582
<class 'str'>
2764
<class 'str'>
3944
<class 'str'>
2744
<class 'str'>
2338
<class 'str'>
5208
<class 'str'>
3645
<class 'str'>
1169
<class 'str'>
61
<class 'str'>
5795
<class 'str'>
53
<class 'str'>
11072
<class 'str'>
289
<class 'str'>
239
<class 'str'>
972
<class 'str'>
187
<class 'str'>
1889
<class 'str'>
3762
<class 'str'>
13332
<class 'str'>
5276
<class 'str'>
3368
<class 'str'>
6683
<class 'str'>
3103
<class 'str'>
3700
<class 'str'>
6639
<class 'st

C:\Users\lukea\AppData\Local\Temp/ipykernel_12376/3107904381.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_length'] = text_length


In [119]:

clean_df.dtypes


name             object
description      object
provider         object
datePublished    object
text             string
text_length       int64
dtype: object

In [120]:
#NLTK
import contractions
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [121]:
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lukea\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lukea\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lukea\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [122]:
test_text = clean_df.loc[:,'text']
test_text

0     NEW You can now listen to Fox News articles!

...
1                                                      
2     NEW You can now listen to Fox News articles!

...
3     © Provided by Washington Examiner

The move by...
4     Florida has been the center of national attent...
                            ...                        
77    In this installment of the FiveThirtyEight Pol...
78    On March 28, Florida Governor Ron DeSantis sig...
79    Placeholder while article actions load

In rec...
80    JACKSONVILLE, Fla. — Gov. Ron DeSantis signed ...
81    Nearly 70 students at Paxon School for Advance...
Name: text, Length: 82, dtype: string

In [167]:
#tokenizing first row

#sent_tokenize(test_text)
#word_tokenize(test_text)
def filter_text(text):
    fixed_text = contractions.fix(test_text[0])
    words_in_text = word_tokenize(fixed_text)
    new_stop_words = ['!',',','.','?','``',"'",'"','-','[',']',"''",'’','–']
    stop_words = set(stopwords.words("english"))
    stop_words.update(new_stop_words)
    #Removes stop words
    filtered_list = [
        word for word in words_in_text if word.casefold() not in stop_words
    ]
    return filtered_list


In [169]:
def flatten_list(lst):
    flat_list = [inner for item in lst for inner in item]
    return flat_list
    

In [171]:
filtered_text = [filter_text(text) for text in test_text.tolist()]
filtered_list = flatten_list(filtered_text)
print(len(test_text),len(filtered_list))

82 24846


In [172]:
# for word in words_in_quote:
#     if word.casefold() not in stop_words:
#         filtered_list.append(word)

#### Stemming

In [174]:
stemmer = PorterStemmer()
stemmed_words = [stemmer.stem(word) for word in filtered_list]
#stemmed_words

#### Lemmatizing

In [175]:
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize("scarves")

'scarf'

In [176]:
lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_list]
#lemmatized_words
#disney + employee being a bigram
#do n't say gay

In [177]:
import collections
import numpy as np
frequency = collections.Counter(filtered_list)
#frequency.most_common()
labels, values = zip(*frequency.items())
indexes = np.arange(len(labels))
df = pd.DataFrame({values}, index=labels)
frequency.most_common()

[('kids', 574),
 ('said', 410),
 ('teachers', 410),
 ('vocal', 410),
 ('pushing', 328),
 ('Weingarten', 328),
 ('parental', 328),
 ('parents', 328),
 ('children', 328),
 ('Randi', 246),
 ('union', 246),
 ('Michon', 246),
 ("'s", 246),
 ('school', 246),
 ('permission', 246),
 ('talk', 246),
 ('AFT', 246),
 ('help', 246),
 ('want', 246),
 ('Fox', 164),
 ('News', 164),
 ('father', 164),
 ('back', 164),
 ('American', 164),
 ('Federation', 164),
 ('Teachers', 164),
 ('President', 164),
 ('need', 164),
 ('rights', 164),
 ('colluded', 164),
 ('America', 164),
 ('years', 164),
 ('agenda', 164),
 ('learn', 164),
 ('slips', 164),
 ('one', 164),
 ('legislation', 164),
 ('Florida', 164),
 ('way', 164),
 ('involvement', 164),
 ('education', 164),
 ('minorities', 164),
 ('marginalize', 164),
 ('censor', 164),
 ('best', 164),
 ('us', 164),
 ('literacy', 164),
 ('minority', 164),
 ('NEW', 82),
 ('listen', 82),
 ('articles', 82),
 ('Loudoun', 82),
 ('County', 82),
 ('Virginia', 82),
 ('defending', 82),